# Un problema de regresión polinomial

En este taller estudiarás conceptos mostrados en el tutorial "Solución de problemas con regresión polinomial". Particularmente, realizarás los siguientes procesos:

1. Cargar un conjunto de datos.
2. Preparar los datos para el modelado.
3. Realizar una búsqueda de hiperparámetros para un modelo de regresión polinomial.
4. Evaluar el mejor modelo resultante.

El problema que vas a resolver es el siguiente: dadas las propiedades acústicas de una canción, predecir su popularidad. Antes de iniciar, importaremos las librerías necesarias:

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, RobustScaler
from sklearn.pipeline import make_pipeline

from importlib.metadata import version

print(f"Versión de Pandas: {version('pandas')}")
print(f"Versión de Seaborn: {version('seaborn')}")
print(f"Versión de Matplotlib: {version('matplotlib')}")
print(f"Versión de Scikit-learn: {version('scikit-learn')}")
print(f"Versión de Numpy: {version('numpy')}")

Versión de Pandas: 2.2.3
Versión de Seaborn: 0.13.2
Versión de Matplotlib: 3.10.0
Versión de Scikit-learn: 1.6.1
Versión de Numpy: 1.25.2


## 1. Carga de datos

Con las librerías importadas, realizaremos la carga del conjunto de datos:

### Ejercicio 1.1.

Utiliza Pandas para importar el archivo que contiene el conjunto de datos de canciones.

* La ruta del archivo .csv es: `./data/song_data.csv`, y ya se encuentra en el entorno de Coursera, solo debes importarlo.
* La variable resultante debe tener el nombre `data_raw`, que representa el conjunto de datos sin modificar.

In [4]:
ruta = './data/song_data.csv'
# your code here
data_raw = pd.read_csv(ruta)

In [5]:
#---------- Celda de Pruebas ----------
# El resultado existe
# El resultado es un DataFrame
# El resultado tiene las dimensiones correctas
#--------------------------------------

# Se verifica que la variable exista
assert data_raw is not None, "Asegúrate de definir la variable /`data_raw/` igualándola a una función de Pandas para leer el archivo."

# Se verifica que sea un DataFrame
assert isinstance(data_raw, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se evalúan las dimensiones de la variable data
assert data_raw.shape == (18835,15), "¿Verificaste que la ruta del archivo CSV y el nombre de la variable son correctos?"
print("¡Los datos tienen las dimensiones correctas!")

¡Los datos tienen las dimensiones correctas!


## 2. Preparación de los datos

Utilizaremos la variable `data` para almacenar un conjunto de datos modificado. Recuerda que usarás esta nueva variable para definir los conjuntos de entrenamiento y pruebas:

In [6]:
data = data_raw.copy()

In [7]:
data.head()

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


In [8]:
data.duplicated().sum()

3909

Es conveniente eliminar los datos duplicados, por lo que utilizaremos la función `drop_duplicates()`:

In [9]:
data = data.drop_duplicates()

In [10]:
data.duplicated().sum()

0

Ahora eliminaremos la columna `song_name` de la variable `data`:

In [11]:
data = data.drop(['song_name'], axis=1)

Y observaremos el resultado con `data.head()`:

In [12]:
data.head()

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


### Ejercicio 2.1.

Inicialmente vamos a separar la variable objetivo `song_popularity` de las variables independientes. Utiliza la variable `data` para construir dos conjuntos de variables:

* Crea una variable con el nombre `x`, que contenga únicamente las variables independientes. (**Ejemplo: `x = data.<<Función>>`**)
* Crea una variable con el nombre `y`, que contenga la variable objetivo. (**Ejemplo: `y = <<Consulta>>`**)

In [13]:
# your code here
x = data.drop(["song_popularity"], axis="columns")
y = data["song_popularity"]


In [14]:
#---------- Celda de Pruebas ----------
# Las variables "x" y "y" existen
# La variable "x" es un DataFrame
# La variable "y" es una Serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que las variables están definidas
assert x is not None, "Asegúrate de definir la variable \'x\' correctamente."
assert y is not None, "Asegúrate de definir la variable \'y\' correctamente."

# Se verifica que "x" sea un DataFrame
assert isinstance(x, pd.DataFrame), "El resultado debe ser un DataFrame."

# Se verifica que "y" sea una Serie
assert isinstance(y, pd.Series), "El resultado debe ser una Serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x.shape == (14926,13), "\'x\' debe tener el mismo número de filas pero una columna menos que \'data\'."
assert y.shape == (14926,), "\'y\' solamente contiene una columna."
print("¡Los conjuntos \'x\' y \'y\' tienen las dimensiones correctas!")

¡Los conjuntos 'x' y 'y' tienen las dimensiones correctas!


### Ejercicio 2.2.

Usando el 80% de los datos para entrenar el modelo y el 20% restante para probarlo, utiliza `scikit-learn` para separar los conjuntos definidos por las variables `x` y `y`.

* Guarda tu respuesta en cuatro variables: `x_train`, `x_test`, `y_train` y `y_test`. (**Ejemplo: `x_train, x_test, y_train, y_test = <<Función>>`**)
* Utiliza el parámetro `random_state=0`. Esto hará que la partición sea siempre la misma.

In [15]:
# your code here
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.2, random_state=0)

In [16]:
#---------- Celda de Pruebas ----------
# Las variables "x_train" y "x_test" existen
# Las variables "y_train" y "y_test" existen
# Las variables "x_train" y "x_test" son un DataFrame
# Las variables "y_train" y "y_test" son una serie de Pandas
# Las variables tienen las dimensiones correctas
#--------------------------------------

# Se verifica que "x_train" y "x_test" están definidas
assert x_train is not None, "Asegúrate de definir la variable \'x_train\' con el nombre correcto."
assert x_test is not None, "Asegúrate de definir la variable \'x_test\' con el nombre correcto."

# Se verifica que "y_train" y "y_test" están definidas
assert y_train is not None, "Asegúrate de definir la variable \'y_train\' con el nombre correcto."
assert y_test is not None, "Asegúrate de definir la variable \'y_test\' con el nombre correcto."

# Se verifica que "x_train" y "x_test" sean un arreglo de Numpy
assert isinstance(x_train, pd.DataFrame), "La variable \'x_train\' debe ser un DataFrame."
assert isinstance(x_test, pd.DataFrame), "La variable \'x_test\' debe ser un DataFrame."

# Se verifica que "y_train" y "y_test" sean un arreglo de Numpy
assert isinstance(y_train, pd.Series), "La variable \'y_train\' debe ser una serie de Pandas."
assert isinstance(y_test, pd.Series), "La variable \'y_test\' debe ser una serie de Pandas."

# Se evalúan las dimensiones de las variables
assert x_train.shape == (11940,13), "Verifica que estés utilizando el 80% de los datos para el conjunto de entrenamiento. El resultado debe tener las mismas columnas que la variable \'x\'."
assert x_test.shape == (2986,13), "Verifica que estés utilizando el 20% de los datos para el conjunto de pruebas. El resultado debe tener las mismas columnas que la variable \'x\'."
assert y_train.shape == (11940,), "Verifica que estés utilizando el 80% de los datos para el conjunto de entrenamiento. El resultado debe ser una sola columna."
assert y_test.shape == (2986,), "Verifica que estés utilizando el 20% de los datos para el conjunto de pruebas. El resultado debe ser una sola columna."
print("¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!")

¡Los conjuntos de entrenamiento y pruebas tienen las dimensiones correctas!


## 3. Búsqueda de hiperparámetros y entrenamiento del modelo

Con el conjunto de datos preparado, es momento de entrenar el modelo de regresión polinomial multivariable. Primero vamos a definir una variable con el nombre `polynomial_regression`, que utilizarás para realizar la búsqueda de hiperparámetros:

In [17]:
polynomial_regression = make_pipeline(
    PolynomialFeatures(),
    RobustScaler(),
    LinearRegression()
)

Utilizaremos un objeto de la clase `KFold`, con el que definiremos 10 subconjuntos sobre el conjunto de entrenamiento:

In [18]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

### Ejercicio 3.1.

El siguiente paso es definir el espacio de búsqueda del hiperparámetro. Para esto define un diccionario con una tupla, cuya llave sea `polynomialfeatures__degree` y su valor sea el arreglo definido en la variable `valores`:

* Define una variable con el nombre `param_grid` y asígnale la expresión necesaria para crear un diccionario con los datos necesarios. Puedes utilizar la variable `valores` dentro de la expresión. (**Ejemplo: `param_grid = <<Expresión>>`**)

In [19]:
valores = [2, 3]
# your code here
param_grid = {"polynomialfeatures__degree":valores}


In [20]:
#---------- Celda de Pruebas ----------
# La variable "param_grid" existe
# La variable "param_grid" es un diccionario
# La variable tiene la longitud correcta
#--------------------------------------

# Se verifica que "param_grid" está definida
assert param_grid is not None, "Asegúrate de definir la variable \'param_grid\' correctamente."

# Se verifica que "param_grid" sea un diccionario
assert isinstance(param_grid, dict), "El resultado debe ser un diccionario de Python, es decir, no necesitas ninguna función específica de ninguna librería para definir la variable."

# Se evalúa la longitud de la variable
assert len(param_grid) == 1, "\'param_grid\' solo debe contener una tupla."
print("¡\'param_grid\' está definida correctamente!")

¡'param_grid' está definida correctamente!


### Ejercicio 3.2.

Finalmente, el último paso antes de realizar la búsqueda de hiperparámetros es crear el objeto de tipo `GridSearchCV`. Utiliza la variable `polynomial_regression`, además de las variables `param_grid` y `kfold` para definirlo:

* Define una variable con el nombre `grid` y asígnale la función necesaria para crear un objeto de la clase `GridSearchCV`. (**Ejemplo: `grid = <<Función>>`**)

In [24]:
# your code here

grid = GridSearchCV(polynomial_regression, param_grid, cv=kfold,n_jobs=-1)

In [25]:
#---------- Celda de Pruebas ----------
# La variable "grid" existe
# La variable "grid" es un objeto de la clase GridSearchCV
#--------------------------------------

# Se verifica que "grid" está definida
assert grid is not None, "Asegúrate de definir la variable \'grid\' correctamente."

# Se verifica que "grid" es un objeto de la clase GridSearchCV
assert type(grid) == GridSearchCV, "La variable \'grid\' debe ser un objeto de la clase GridSearchCV."
print("¡\'grid\' está definida correctamente!")

¡'grid' está definida correctamente!


### Ejercicio 3.3.

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables `x_train` y `y_train`.

* Para este ejercicio no debes asignar tu resultado a ninguna variable. Es decir, solo debes ejecutar una función sobre la variable `grid`, utilizando las variables `x_train` y `y_train` como parámetros. (**Ejemplo: `grid.<<Función>>`**)

In [26]:
# your code here
grid.fit(x_train, y_train)


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('robustscaler', RobustScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             n_jobs=-1, param_grid={'polynomialfeatures__degree': [2, 3]})

In [27]:
#---------- Celda de Pruebas ----------
# El atributo "best_params_" de la variable "grid" existe
# El atributo "best_estimator_" de la variable "grid" existe
#--------------------------------------

# El atributo "best_params_" está definido
assert grid.best_params_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un diccionario con el mejor hiperparámetro."

# El atributo "best_estimator_" está definido
assert grid.best_estimator_ is not None, "Asegúrate de ejecutar la función de entrenamiento para generar un modelo de regresión polinomial."

# Se verifica que "grid" haya generado un solo mejor hiperparámetro
assert len(grid.best_params_) == 1, "Al ejecutar una función usando el objeto \'grid\', se debe generar una sola tupla con el hiperparámetro."
print("¡Se ha realizado la búsqueda de hiperparámetros correctamente!")

¡Se ha realizado la búsqueda de hiperparámetros correctamente!


Podemos observar el mejor hiperparámetro usando el atributo `best_params_`:

In [28]:
print("Mejor parámetro: ", grid.best_params_)

Mejor parámetro:  {'polynomialfeatures__degree': 2}


Además, vamos a definir la variable `mejor_modelo` para realizar las pruebas:

In [29]:
mejor_modelo = grid.best_estimator_

## 4. Evaluación del modelo

Por último, evaluarás el modelo entrenado utilizando el conjunto de pruebas.

### Ejercicio 4.1.

Realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.

* Utiliza el objeto `mejor_modelo` para realizar las predicciones. Asigna el resultado a una variable con nombre `y_pred` (**Ejemplo: `y_pred = mejor_modelo.<<Función>>`**).
* Encontrarás una línea solo con el nombre de la variable. Esta línea se usa para que puedas visualizar tu respuesta, por lo que siempre debe ir al final y no la debes modificar.

In [32]:
# your code here

y_pred = mejor_modelo.predict(x_test)

In [33]:
#---------- Celda de Pruebas ----------
# La variable "y_pred" existe
# La variable "y_pred" es un arreglo
# La variable "y_pred" tiene las dimensiones correctas
#--------------------------------------

# Se verifica que "y_pred" está definida
assert y_pred is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "y_pred" sea un arreglo
assert isinstance(y_pred, np.ndarray), "El resultado debe ser un Arreglo de Numpy."

# Se evalúan las dimensiones de "y_pred"
assert y_pred.shape == (2986,), "\'y_pred\' debe tener el mismo número de filas (predicciones) que \'x_test\' y \'y_test\'."
print("¡\'y_pred\' es un arreglo con las dimensiones correctas!")

¡'y_pred' es un arreglo con las dimensiones correctas!


### Ejercicio 4.2.

Finalmente, obtén tres métricas de rendimiento utilizando las predicciones del modelo.

#### Ejercicio 4.2.1.

Utiliza `scikit-learn` para obtener la Raíz del Error Cuadrático Medio (RMSE).

* Define una variable con el nombre `rmse` y asígnale la función necesaria para obtener el RMSE (`rmse = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [36]:
# your code here
rmse = root_mean_squared_error(y_test, y_pred)
print("RMSE: ", rmse)


RMSE:  20.01243359190176


In [37]:
#---------- Celda de Pruebas ----------
# La variable "rmse" existe
# La variable "rmse" es un número
# La variable "rmse" tiene un valor válido
#--------------------------------------

# Se verifica que "rmse" está definida
assert rmse is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "rmse" sea un número
assert isinstance(rmse, np.float64) or isinstance(rmse, float), "El resultado debe ser un número decimal."

# Se verifica que "rmse" tiene un valor válido
assert rmse >= 0 and rmse <= 100, "\'rmse\' debe tener un valor entre 0 y 100 (los valores posibles de la popularidad)."
print("¡\'rmse\' es un número válido!")

¡'rmse' es un número válido!


#### Ejercicio 4.2.2.

Utiliza `scikit-learn` para obtener el Error Absoluto Medio (MAE).

* Define una variable con el nombre `mae` y asígnale la función necesaria para obtener el MAE (`mae = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [38]:
# your code here
mae= mean_absolute_error(y_test, y_pred)

print("MAE: ", mae)

MAE:  15.97864692092859


In [ ]:
#---------- Celda de Pruebas ----------
# La variable "mae" existe
# La variable "mae" es un número
# La variable "mae" tiene un valor válido
#--------------------------------------

# Se verifica que "mae" está definida
assert mae is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "mae" sea un número
assert isinstance(mae, np.float64) or isinstance(mae, float), "El resultado debe ser un número decimal."

# Se verifica que "mae" tiene un valor válido
assert mae >= 0 and mae <= 100, "\'mae\' debe tener un valor entre 0 y 100 (los valores posibles de la popularidad)."
print("¡\'mae\' es un número válido!")

#### Ejercicio 4.2.3.

Utiliza `scikit-learn` para obtener el Coeficiente de determinación (R<sup>2</sup>).

* Define una variable con el nombre `r2` y asígnale la función necesaria para obtener el R<sup>2</sup> (`r2 = <<Función>>`).
* Encontrarás una línea que mostrará tu resultado. Esta línea debe ir al final y no la debes modificar.

In [39]:
# your code here
r2 =r2_score(y_test, y_pred)
print('R²: ', r2)
print(f'R²: {r2_score(y_test, y_pred):.2f}')

R²:  0.051570907523971976
R²: 0.05


In [40]:
#---------- Celda de Pruebas ----------
# La variable "r2" existe
# La variable "r2" es un número
# La variable "r2" tiene un valor válido
#--------------------------------------

# Se verifica que "r2" está definida
assert r2 is not None, "Asegúrate de definir la variable con el nombre correcto."

# Se verifica que "r2" sea un número
assert isinstance(r2, np.float64) or isinstance(r2, float), "El resultado debe ser un número decimal."

# Se verifica que "r2" esté en el rango correcto
assert r2 >= 0 and r2 <= 1, "El resultado debe estar entre 0 y 1."
print("¡\'r2\' es un número válido!")

¡'r2' es un número válido!


## Cierre

Al desarrollar los ejercicios de este taller, has practicado tus capacidades para modificar un conjunto de datos y realizar una búsqueda de hiperparámetros. Adicionalmente, realizaste predicciones con el mejor modelo de regresión polinomial y calculaste tres métricas de rendimiento sobre los resultados.

---
*Creado por: Nicolás Díaz*

*Última edición: Camilo Rozo*

*Revisado por: Haydemar Nuñez*

*Versión: Enero 2025*  

*Universidad de los Andes* 